## Gradient Boosting Model on Weather Station Data + Previous ergot engineer features

- Gradient builds an additive model in a forward stage-wise fashion, which allows for the optimization of arbitrary differentiable loss functions. 

In [1]:
# import dependencies
import pandas as pd
import sqlalchemy as sq
import sys, os
import pickle
from imblearn.combine import SMOTEENN
from xgboost import XGBClassifier
from sklearn.ensemble import (  # type: ignore
    GradientBoostingClassifier,
)
from imblearn.ensemble import (  # type: ignore
    RUSBoostClassifier,
)

from sklearn.metrics import (  # type: ignore
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

sys.path.append("../../")
os.chdir("../../")
from ModelBuilderMethods import getConn, extractYears

In [ ]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

### <u>**Step 1**</u>: Data Selection

In this step, we would choose the particular data/table, pick attributes from existing tables. Further aggregation/feature engineer can be done here to support the point of the research.

Particular, for this notebook, we grab the following data and merge them (on year, district) into a single table:
- Weather station data
- ergot data (downgrade)
- ergot previous feature engineer

In [3]:
weatherStationQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_station
"""
)

ergotPrevYearsAggQuery = sq.text(
    """
    SELECT year, district, 
    present_prev1, present_prev2, present_prev3,
    percnt_true_prev1, percnt_true_prev2, percnt_true_prev3 
    from agg_ergot_sample_v2
"""
)

ergotTargetQuery = sq.text(
    """
    SELECT year, district, downgrade from ergot_sample_feat_eng
"""
)

In [4]:
conn = getConn("./.env")

stationDf = pd.read_sql(weatherStationQuery, conn)
ergotPrevDf = pd.read_sql(ergotPrevYearsAggQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [5]:
# merge on year and district
tempdf = pd.merge(stationDf, ergotPrevDf, on=["year", "district"], how="left")
del stationDf
del ergotPrevDf
# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

In [6]:
# encode district
datasetDf["district"] = datasetDf["district"].astype("category")

temp = pd.get_dummies(datasetDf["district"], prefix="district", drop_first=True)
datasetDf = pd.concat([datasetDf, temp], axis=1)

datasetDf = datasetDf.drop(columns=["district"])

del temp

In [7]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

In [8]:
# drop year
trainDf = trainDf.drop(columns=["year"])
testDf = testDf.drop(columns=["year"])

In [9]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

downgrade
False    122202
True       2082
Name: count, dtype: int64
downgrade
False    26307
True      1016
Name: count, dtype: int64


In [10]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
# trainDf = trainDf.fillna(0)

# drop nan
trainDf = trainDf.dropna()

downgrade                     0
1:min_temp_x               1246
1:max_temp_x               1246
1:mean_temp_x              1246
1:min_dew_point_temp       1246
1:max_dew_point_temp       1246
1:mean_dew_point_temp      1246
1:min_humidex              1246
1:max_humidex              1246
1:mean_humidex             1246
1:min_precip               1246
1:max_precip               1246
1:mean_precip              1246
1:min_rel_humid            1246
1:max_rel_humid            1246
1:mean_rel_humid           1246
1:min_stn_press            1246
1:max_stn_press            1246
1:mean_stn_press           1246
1:min_visibility           1246
1:max_visibility           1246
1:mean_visibility          1246
1:max_temp_y               1246
1:min_temp_y               1246
1:mean_temp_y              1246
1:min_total_rain           1246
1:max_total_rain           1246
1:mean_total_rain          1246
1:min_total_snow           1246
1:max_total_snow           1246
1:mean_total_snow          1246
1:min_to

In [11]:
balancer = SMOTEENN(sampling_strategy=1, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(
    trainDf.drop(columns="downgrade"), trainDf["downgrade"]
)

In [12]:
# post balancing check
# print value counts downgrade
print(balancedTrainDfY.value_counts())

downgrade
False    76741
True     15427
Name: count, dtype: int64


In [13]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

### <u>**Step 5**</u>: Gradient Boosting Classifier Model

##### <u>**Step 5.1**</u>: Initialize the model

In [14]:
ESTIMATORS = 400
DEPTH = 40
CORES = -1
MINSPLSPLIT = 8
MINSAMPLELEAF = 4

gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
balanced_gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
balanced_rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)
balanced_xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)

##### <u>**Step 5.2**</u>: Fit the training data to the model

In [15]:
gradient_boosting_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_gradient_boosting_model.fit(balancedTrainDfX, balancedTrainDfY)

      Iter       Train Loss   Remaining Time 
         1           0.1933           39.75m
         2           0.1874           39.63m
         3           0.1832           39.40m
         4           0.1800           39.31m
         5           0.1774           39.19m
         6           0.1754           39.12m
         7           0.1736           38.96m
         8           0.1722           38.83m
         9           0.1709           38.55m
        10           0.1699           38.30m
        20           0.1646           36.46m
        30           0.1630           35.07m
        40           0.1625           33.67m
        50           0.1623           32.21m
        60           0.1622           30.74m
        70           0.1622           29.35m
        80           0.1622           28.04m
        90           0.1622           26.82m
       100           0.1622           25.62m
       200           0.1622           14.67m
      Iter       Train Loss   Remaining Time 
        

GradientBoostingClassifier(max_depth=40, n_estimators=400, n_iter_no_change=200,
                           random_state=42, verbose=1)

In [18]:
rusboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_rusboost_model.fit(balancedTrainDfX, balancedTrainDfY)

RUSBoostClassifier(n_estimators=400, random_state=42, sampling_strategy=0.5)

In [20]:
xgboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_xgboost_model.fit(balancedTrainDfX, balancedTrainDfY)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:present_prev1: object, present_prev2: object, present_prev3: object

In [16]:
testDf = testDf.dropna()

In [21]:
# get predictions

predictions_gradient_boosting = gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_balanced_gradient_boosting = balanced_gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_rusboost = rusboost_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_rusboost = balanced_rusboost_model.predict(
    testDf.drop(columns="downgrade")
)
# predictions_xgboost = xgboost_model.predict(testDf.drop(columns="downgrade"))
# predictions_balanced_xgboost = balanced_xgboost_model.predict(
#     testDf.drop(columns="downgrade")
# )

In [22]:
print(pd.DataFrame(predictions_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_balanced_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_rusboost).value_counts())
print(pd.DataFrame(predictions_balanced_rusboost).value_counts())
# print(pd.DataFrame(predictions_xgboost).value_counts())
# print(pd.DataFrame(predictions_balanced_xgboost).value_counts())

False    27295
Name: count, dtype: int64
False    17373
True      9922
Name: count, dtype: int64
False    22617
True      4678
Name: count, dtype: int64
False    24995
True      2300
Name: count, dtype: int64


##### <u>**Step 5.4**</u>: Evaluate models based on different metrics:
- ACCURACY:
- PRECISION:
- RECALL:
- F1:
- ROC AUC:

In [23]:
# get accuracy precision recall f1 roc_auc
printMetrics(
    "sk GB imbalanced train set", testDf["downgrade"], predictions_gradient_boosting
)
printMetrics(
    "imb GB balanced train set",
    testDf["downgrade"],
    predictions_balanced_gradient_boosting,
)
printMetrics("sk RUS imbalanced train set", testDf["downgrade"], predictions_rusboost)
printMetrics(
    "imb RUS balanced train set", testDf["downgrade"], predictions_balanced_rusboost
)
# printMetrics("sk XGB imbalanced train set", testDf["downgrade"], predictions_xgboost)
# printMetrics(
#     "imb XGB balanced train set", testDf["downgrade"], predictions_balanced_xgboost
# )

sk GB imbalanced train set
Accuracy:  0.9627770653965928
Precision:  0.0
Recall:  0.0
F1:  0.0
ROC AUC:  0.5
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      1.00      0.98     26279
        True       0.00      0.00      0.00      1016

    accuracy                           0.96     27295
   macro avg       0.48      0.50      0.49     27295
weighted avg       0.93      0.96      0.94     27295


imb GB balanced train set
Accuracy:  0.630921414178421
Precision:  0.0435396089498085
Recall:  0.4251968503937008
F1:  0.07899067471201317
ROC AUC:  0.5320359989249223
Classification Report: 
               precision    recall  f1-score   support

       False       0.97      0.64      0.77     26279
        True       0.04      0.43      0.08      1016

    accuracy                           0.63     27295
   macro avg       0.50      0.53      0.42     27295
weighted avg       0.93      0.63      0.74     27295


sk RUS imbalanced

/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa